### This file contains a personal Google API key of one of the teammates. We have masked this key so this code cannot be executed.
### However, we have added the results of the code in the files of this folder.

In [ ]:
my_apikey = ''

In [0]:
#import file is process with https://geoservices.tamu.edu/default.aspx 
#both enyc_geocoded and ny2_geo are both output files
#input for TAMU's geo service is provided as TAMU_geolocation_example.csv

import pandas as pd
geo = pd.read_csv('nyc_geocoded.csv')
geo2 = pd.read_csv('ny2_geo.csv')

In [0]:
!pip install gmplot 

     |████████████████████████████████| 122kB 10.2MB/s 
  Created wheel for gmplot: filename=gmplot-1.2.0-cp36-none-any.whl size=143765 sha256=f19ed1dce50a8c745cbf8a4e0f0eb57c534fd922777c56d246c33f37785f84cc
  Stored in directory: /root/.cache/pip/wheels/81/6a/76/4dd6a7cc310ba765894159ee84871e8cd55221d82ef14b81a1
Successfully built gmplot


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=1H1rKEThhq0Yf6UgjW9b1t6VmE53moe6M'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
import pandas as pd
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df3 = pd.read_csv('Filename.csv')
df = df3[df3['Violation County']=='NY']
df ['address'] = df['House Number'] +' ' + df['Street Name']+ ' NYC'
df = df[:5001]

1H1rKEThhq0Yf6UgjW9b1t6VmE53moe6M


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
geo2 = geo2.rename(columns={'address': "Address"})
geo2

,NoName,Address,city,State,zipcode,AUTO_UNIQUE_ID_2019-12-02_dandi_ny2,Source,TimeTaken,UpdatedGeocoding,Version,ErrorMessage,TransactionId,naaccrQualCode,naaccrQualType,FeatureMatchingResultType,MatchedLocationType,RegionSizeUnits,InterpolationType,RegionSize,InterpolationSubType,FeatureMatchingGeographyType,MatchScore,FeatureMatchingHierarchy,TieHandlingStrategyType,FeatureMatchingResultTypeTieBreakingNotes,GeocodeQualityType,FeatureMatchingHierarchyNotes,FeatureMatchingResultTypeNotes,FeatureMatchingResultCount,Latitude,Longitude,MatchType
0,14792.0,21 E 7th St NYC,New York City,NY,NaN,1.0,USCGeocoder,828.0700,1.0,4.1,NaN,NaN,0.0,AddressPoint,Ambiguous,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,2.0,40.728653,-73.989462,Relaxed;Soundex
1,14806.0,1177 6th Ave NYC,New York City,NY,NaN,2.0,USCGeocoder,62.4615,1.0,4.1,NaN,NaN,3.0,StreetSegmentInterpolation,Success,LOCATION_TYPE_STREET_ADDRESS,Meters,LinearInterpolation,1883.958209,LinearInterpolationAddressRange,StreetSegment,96.005917,FeatureClassBased,ReturnAll,NaN,AddressRangeInterpolation,NaN,NaN,1.0,40.757367,-73.982383,Relaxed
2,14818.0,1 Penn Plz NYC,New York City,NY,NaN,3.0,USCGeocoder,562.2539,1.0,4.1,NaN,NaN,0.0,AddressPoint,Ambiguous,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,2.0,40.750900,-73.991600,Relaxed
3,14819.0,60 E 96th St NYC,New York City,NY,NaN,4.0,USCGeocoder,562.5023,1.0,4.1,NaN,NaN,0.0,AddressPoint,Ambiguous,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,2.0,40.786843,-73.953659,Relaxed;Soundex
4,14821.0,1360 3rd Ave NYC,New York City,NY,NaN,5.0,USCGeocoder,203.1154,1.0,4.1,NaN,NaN,0.0,AddressPoint,Success,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,1.0,40.773233,-73.958288,Relaxed;Soundex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,29478.0,273 8th Ave NYC,New York City,NY,NaN,2493.0,USCGeocoder,187.5380,1.0,4.1,NaN,NaN,0.0,AddressPoint,Success,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,1.0,40.745771,-73.998369,Relaxed;Soundex
2493,29482.0,2246 3rd Ave NYC,New York City,NY,NaN,2494.0,USCGeocoder,203.3735,1.0,4.1,NaN,NaN,0.0,AddressPoint,Success,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,1.0,40.801942,-73.937324,Relaxed;Soundex
2494,29491.0,109 Mercer St NYC,New York City,NY,NaN,2495.0,USCGeocoder,531.0151,1.0,4.1,NaN,NaN,0.0,AddressPoint,Ambiguous,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,2.0,40.723800,-73.999400,Relaxed
2495,29496.0,80 Maiden Ln NYC,New York City,NY,NaN,2496.0,USCGeocoder,734.2516,1.0,4.1,NaN,NaN,0.0,AddressPoint,Ambiguous,LOCATION_TYPE_STREET_ADDRESS,Meters,ArealInterpolation,0.000000,ArealInterpolationGeometricCentroid,Parcel,97.337278,FeatureClassBased,ReturnAll,NaN,AddressPoint,NaN,NaN,2.0,40.707100,-74.007500,Relaxed


In [0]:
#combine
df_com1 = df.merge(geo, left_on='address',right_on = 'Address' ,how = 'inner')
df_com2 = df.merge(geo2, left_on='address',right_on = 'Address' ,how = 'inner')
dfdf = pd.concat([df_com1,df_com2])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# dfdf['Violation Code'].value_counts()
dfdf['Violation Code'].iloc[0] == 47

True

In [0]:
violist = list(df.groupby(['Violation Code'])['Violation Code'].count().sort_values(ascending = False)[:6].index)

In [0]:
dfdf['Violation Code'] = dfdf['Violation Code'].apply(lambda x: str(x))

In [0]:
dfdf = dfdf.dropna(subset = ['Longitude','Latitude'],how='any')

In [0]:
#produce colored scatter plot
import gmplot 
gmap3 = gmplot.GoogleMapPlotter(40.729552, -73.992197, 13,apikey=my_apikey )
colorlist = ['#FF0000','#ed7f09','#ffcc00','#4ec730','#3053c7','#8a1b9e']
count = 0
for i in range(0,len(violist)):
  x = dfdf[dfdf['Violation Code']==violist[i]]
  lat = list(x['Latitude'])
  lon = list(x['Longitude'])
  color = str(colorlist[i])
  count+=len(lat)
  gmap3.scatter(lat, lon, color, size = 10, marker = False, edge_width=4)
gmap3.draw( "/content/colored.html" ) 

### The following is the code to produce overall violation of NY

In [0]:
lat_lon = geo[['Latitude','Longitude']]
lat_lon = lat_lon.dropna(how='any')
lat_lon2 = geo2[['Latitude','Longitude']]
lat_lon2 = lat_lon2.dropna(how='any')

In [0]:
import gmplot 
lat_list = list(lat_lon['Latitude'])
lon_list = list(lat_lon['Longitude'])
lat_list2 = list(lat_lon2['Latitude'])
lon_list2 = list(lat_lon2['Longitude'])

lat = lat_list+lat_list2
lon = lon_list+lon_list2
gmap3 = gmplot.GoogleMapPlotter(40.729552, -73.992197, 13,apikey=my_apikey )
gmap3.scatter( lat, lon, '#FF0000', size = 10, marker = False,edge_width=4)



gmap3.apikey = my_apikey
gmap3.draw( "/content/map.html" ) 
